# Supplementary Labsheet 3: Inertial Sensors and Magnetometer

**From previous experience, the IMU is known to have a relatively high degree of noise. This means that you are likely to get a better estimate of position from your encoders, kinematics and odometry.**

**For Assessment 2, it is permitted to use the software libraries detailed in this labsheet.**

This supplementary labsheet introduces the IMU (accelerometer, gyro).  These are advanced devices for your 3Pi+.  They can be used to help track your heading, any rotation, and any external influences on the robot pose.

To use these sensor, we will use the `Wire` library from Arduino, which will facilitate communication with the sensors.  These are digital sensors that communicate via the `i2c` protocol ("i squared c", <a href="https://en.wikipedia.org/wiki/I%C2%B2C">inter-integrated circuit</a>).  Each sensor has an address, and from the 32u4 microcontroller we request to write settings or read values.  We will also use the `LSM6` library provided by Pololu to operate the acceleromter and gyroscope.  We will also use the `LIS3MDL` library from Pololu to operate the magenetometer.  These libraries will provide the functionality to read from the sensors, but we will still need to develop code to process the sensor readings.

Each of the devices has key advantages and disadvantages.  You should think carefully about in which applications (`task`, `environment`) these sensors might be useful.  These sensors will likely require significant development time to get them working and to verify them.

This labsheet will cover the following main topics:
- Getting Data from the Intertial Measurement Unit
- Calibrating Sensors
- Filtering Sensors
- Simple Sensor Fusion

**Note:** It appears that opening and closing the Serial port when using these sensors causes the 3Pi+ robot to reset.  It may take a second or two for the reset process to complete before you can open a Serial port again.


<br><br><br>

# Inertial Measurement Unit (IMU)

The 3Pi+ has two chips set out in the schematic as the IMU:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/3PI_IMU_schem.png?raw=true">
</p>

In the above, we can see the chips `LIS3MDL` and `LSM6DS33`.  A quick search on the internet reveals that the LIS3MDL is the magnetometer (<a href="https://www.st.com/resource/en/datasheet/lis3mdl.pdf">datasheet</a>) whilst the LSM6DS33 is an gyro and accelerometer integrated into a single package (<a href="https://cdn.ozdisan.com/ETicaret_Dosya/615475_5463640.pdf">datasheet</a>).  

When we look at the above schematic, these i2c chips will communicate with the 32u4 along the connections made to the labels `SCL` and `SDA`.

`SCL` stands for `Serial Clock`, and it is used to make sure that the microcontroller and peripheral device (here, out sensors) will read and write bits transmitted at the same time.  `SDA` stands for `Serial Data`, and it is the connection that is used to indicate a `0` or `1` being transmitted between the devices.  

Many devices can be connected to the two pins of the 32u4 functioning as `SCL` and `SDA`.  When many devices are connected together in this way, the connection is referred to as a `bus`.  To allow each device to operate independently, each device should have a unique address.  Therefore, the 32u4 will first of all transmit the address of the device we wish to operate.


We can locate these chips on the control circuit board of the 3Pi+:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/3PI_IMU_PCB.png?raw=true">
</p>

In the above, you can also see that Pololu have conveniently printed in white labels to indicate the orientation of the sensor devices.  We can see that the devices have their x axis orientated front-to-back, y axis left-to-right, and the z axis vertically through the robot.  However, what these axis means for each sensor varies.

An accelerometer measures acceleration, or changes in velocity. A gyroscope measures angular velocity.  A magnetometer measures magnetic field strength.  Each of these can read from the 3 different axis directions simultaneously.

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/IMU_Axes.png?raw=true"/>
</p>


It is important to note that, for the accelerometer, the force will be measured through (or along) the axis.  For the gyro and magnetometer, the measurements are taken as rotation about the axis.

<hr><br><br><br><br>



## Exercise 1: Getting data from the Accelerometer and Gyro (LSM6DS33)

Unlike the sensors we have seen so far, this sensor needs to be addressed and communicated with to get readings.  This can seem like a daunting task, however the use of Libraries make sensors such as these relatively simple to operate.

We will install the `LSM6` library from Pololu. Follow these steps:

1. Within the Arduino IDE, click:
  - `Sketch -> Include Library -> Manage Libraries`.

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/Library_screen.png?raw=true"/>  
</p>

2. Search for "LSM6". Make sure you install the Pololu library, you can install the latest version:  

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/Library_Search_Screen.png?raw=true"/>
</p>


3. We can make a quick test of the accelerometer and gyro with the following example code:

```c

// These two includes are necessary to read
// the LSM6DS33 (accelerometer and gyro)
#include <Wire.h>
#include <LSM6.h>

// LSM6 is a class.  We create an instance
// of this class called "imu".  We will then
// use imu to access the device by method
// functions and variables inside the class.
LSM6 imu;

void setup() {
  
  // Start the wire library for i2c.
  // Note: do not add this command into
  // a class constructor. It must occur
  // (or be called) from setup().
  Wire.begin();

  // Serial for debug output
  Serial.begin(9600);
  Serial.println("***RESET***");
  delay(1000);

  // Check the IMU initialised ok.
  if (!imu.init() ) {  // no..? :(

    // Since we failed to communicate with the
    // IMU, we put the robot into an infinite
    // while loop and report the error.
    while (1) {
      Serial.println("Failed to detect and initialize IMU!");
      delay(1000);
    }
  }

  // IMU initialise ok!
  // Set the IMU with default settings.
  imu.enableDefault();

} // End of setup()



// Simple loop to report all the readings from
// the accelerometer and gyro
// imu.a = accelerometer
// imu.g = gyro
void loop() {

  // Make a read of the sensor.
  imu.read();

  // Report values
  Serial.print("A: ");
  Serial.print(imu.a.x);
  Serial.print(" ");
  Serial.print(imu.a.y);
  Serial.print(" ");
  Serial.print(imu.a.z);
  Serial.print("\t G:  ");
  Serial.print(imu.g.x);
  Serial.print(" ");
  Serial.print(imu.g.y);
  Serial.print(" ");
  Serial.println(imu.g.z);

  // Short delay to keep things
  // slow enough to observe.
  // There is a limit to how fast you
  // can make i2c readings.
  delay(100);

  
} // End of loop()
```





<hr><br><br><br><br>

## Exercise 2: Sensor Validation and Configuration

Verify that the IMU readings are sensible. Upload the code above, open the Serial plotter and see what happens as you move the robot around.

You should see that when the robot is placed flat on the ground, you get an accelerometer reading in the Z direction of around 17000. Here, this constant reading through the z-axis is gravity.  How do we interpret this number?

Unlike our previous experience with sensors, the IMU computes and provides this reading autonomously. Because the IMU is a digital sensor, we can change the sensitivity of the sensor to suit our application. If we look in the datasheet, we can find the following table:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/IMU_Conversion_factors.png?raw=true">
</p>

This table tells us the conversion factor for each sensitivity setting. For example, if we set the accelerometer to have a range of +/- 2g, then the least significant bit of the output is equal to 0.061 mg (g here refers to gravity, not grams). This means that if we multiply the output by 0.061, then we will get a measurement in units of mg.

Similarly, for the gyroscope, if we use the default sensitivity (+/-245 mdps, mili degrees per second), we should multiply the readings by 8.75 to get a reading with units of mdps.

Note that in the Sketch above, when we call the line:

```c
 imu.enableDefault();
```

we set the default sensitivity for these sensors (+/-2g for the accellerometer, and +/- 245 mpds for the gyroscope). If we want to, we can select a different sensitivity for each sensor. To do so, we look in the datasheet.

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/A_full_scale_register.png?raw=true">
</p>

This table tells us the meaning of the bits in the linear sensor control register (CTRL1_XL). We can see that the four left most bits are used to control the output data rate, which we are not interested in here. The next two bits (labelled, `FS_XL0` and `FS_XL1`) set the full-scale sensitivity.

If we set `FS_XL0` and `FS_XL1` to 00, then we will have a sensitivity of +/-2g. If we want a greater range, we can do this by changing these values. For example, we could increase the range to +/- 4g by setting these bits to equal 01.

Note that an increased range means we lose resolution (As the sensor can only output information to 16 bit precision). To actually change the `FS_XL0` and `FS_XL1` values within the sensor, we have to send the data with `i2c`. To do this, we use the writeReg function. For example, we could change the accellerometer sensitivity to +/-4g by writing:

```c
 imu.writeReg(LSM6::CTRL1_XL, 0b01011000); // 208 Hz, +/4 g
```

However, we must remember that if we change the sensitivity, then the conversion factor will also change. In this case, we should now multiply our readings by 0.122 to get a unit of mg.

1. Add a conversion factor for the accellerometer and gyroscope and verify that the units are approximately correct (You should see a reading of about 1000 from the accelerometer Z axis).





<hr><br><br><br><br>

## Exercise 3: Calibrating the Gyro

In theory, one of the advantages of a digital sensor is that accurate calibration can be performed at the factory, eliminating the need for us to calibrate it ourselves. In practice, this is not the case and it is still necessary to calibrate many digital sensors.

For example, if we simply place the robot flat on the table and read the Gyroscope's reading around the Z-axis we will find that even though the robot is not moving, it still gives us a non-zero reading.  

To turn this angular rate information into a heading angle, we need to integrate it. If we integrate this raw data, we will get a heading which changes, even though the robot is not moving.

As this is a systematic error, we can remove it by calibrating. To do so, we: Collect a large number of readings, take the mean, and then subtract this value from all future readings.  Be cautious of overflow errors in your variables and type conversions.

1. Implement a calibration routine for the Gyroscope.

2. Integrate the gyroscope heading (You can use a simple summation approach like we did in the Kinematics) to measure the heading of your robot.

3. Use the Gyroscope heading information to create a `ResistRotation` behaviour that tries to maintain a fixed angle, even if the robot is picked up and moved by hand. This is a perfect task for a PID controller, providing speed as the demand into PID speed controllers for left and right wheels.

<hr><br><br><br><br>

# Filtering Sensor Data




## Exercise 4:

If we plot the raw readings from any of our sensors, we will notice that the readings are not stable, even when the robot is not moving. For example, below is the x and y components of the accelerometer reading when the robot is sat on the table:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/Acc_noise.png?raw=true"/>
</p>

In order to get a useful signal out of the sensor, we can apply a filter to reduce this noise. In the case of the accelerometer and magnetometer, the noise is mostly at high-frequency, and so we can remove this by applying a simple low-pass filter.

A simple form of low-pass filter well suited for implementation on a micro-controller is the exponential moving average filter. This produces an output according to:

$$ output_{t} = (\alpha * reading) + ((1-\alpha)*output_{t-1}) $$

Intuitively, we can think of this filter as producing an output which is a combination of the past readings and the current reading. The co-efficient alpha controls how much weight we put on the current reading vs how much we put on the past.  Another way to think about this would be to ask:

> how much does the newest reading contribute to our average?

1. Characterise the noise produced by your magnetomer's heading estimate. With the robot steady on the table, record a series of measurements and note the maximum and minimum values.
  - Investigate the effect of changing alpha on the range of readings. Do you notice any other effects of changing the value of alpha?




<hr><br><br><br><br>

# Simple Sensor Fusion: A Complementary Filter



## Exercise 5:

During your experimentation, you should have seen that although the low-pass filtered heading from the magnetometer is pretty reliable when the robot is stationary.  When the robot begins to move, the heading estimate from the magnetometer does not keep up with the movement.  Because of this, another name for the filter above is the <i>lag</i> filter.  This means that while we can trust the low-pass filter to give us a good estimate in the long term, on short time scales, it may be wrong.

To get a better estimate of the robot's heading, we need to combine the lag filter with a second source of information that is reliable over these short time scales. A simple way of doing this is with a complementary filter.

To fully understand the details of a complementary filter, we would need to introduce frequency domain control design. Intuitively, we can think of a complementary filter as deciding which of two sources of information to trust.

One sensor (the magnetometer) in our case is trusted on long time scales, while another (The gyroscope in this case) is trusted only in the short term. A simple way to implement a complementary filter is with:

$$ heading_t = \alpha * (heading_{t-1} + gyro\_reading * elapsed\_time) + ((1-\alpha)*mag\_reading) $$

We should set alpha in the range [ 0.0 : 1.0 ], typically something higher like `0.9`.  In this case, if the gyro_reading is small (i.e we are not rotating quickly), the heading would converge towards the output of the low pass filter we implemented in the previous exercise.  However, if we do start to move quicky, then the gyro_reading will be integrated and added to the heading, allowing us to respond to quick changes as they are happening.

1. Implement a complentary filter for heading. As before, investigate the effect of varying alpha.







## Exercise 6:

A `G-H filter` estimates a variable (say the heading) as a weighted sum of a prediction and a measurement. We can use a similar idea to combine the information from our encoders with the information from the complementary filter. This should give us a heading estimate which is robust to things like wheel slip and collisions.

To implement such a system on the 3Pi+, we can use the estimate of angular velocity we get from the Kinematics as our "prediction" and then correct this with the measurement from our Complementary filter. The update equations then look like:

$$ speed = (1-h) * speed + (h* angularVelocity) $$
$$ prediction = heading + (speed * elapsed\_time) $$
$$ heading = prediction + g * (complementary\_heading - prediction) $$

Note that the first line of these update equations is effectively low-pass filtering the angular velocity estimate we get from the Kinematics.

1. Implement this update on the 3Pi+. You will need to select appropriate values for the two constants $g$ and $h$. Conduct an experiment to see how varying these parameters effect the output of the filter.

2. Integrate these updates into your kinematic class. You will need to keep the heading calculated purely from the encoder counts, but you can add a complementary_heading and g_h_filter_heading variable to the class and update them with everything else. You can then use the heading angle calculated by the g_h_filter for the position update in your Kinematics.
  - Hint: Remember that both the magnetometer and IMU are updated internally at a set rate. You can find this (and how to change it) in the datasheets. You should ensure your Kinematics are not updated faster than the IMU/Magnetometer.
  


If you wish to pursue GH Filters and sensor fusion further, a good source of information are <a href="https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python#kalman-and-bayesian-filters-in-python">the web articles published by RLabbe on Github.</a>


